<a href="https://colab.research.google.com/github/khadidja2005/Pytorch_practice/blob/main/CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
pip install torchmetrics

In [16]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import os
import torchmetrics
from torchmetrics.classification import Accuracy

In [17]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])


train_dataset = datasets.CIFAR10(root = "/data" , train = True , download = True , transform=transform)
test_dataset = datasets.CIFAR10(root = "/data" , train = False , download = True , transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
]))

Files already downloaded and verified
Files already downloaded and verified


In [18]:
train_loader = DataLoader(train_dataset  , batch_size=128 , shuffle=True , num_workers=2)
test_loader = DataLoader(test_dataset , batch_size=128 , shuffle=False , num_workers=2)

#classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
classes = np.array( train_dataset.classes)
classes

array(['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog',
       'horse', 'ship', 'truck'], dtype='<U10')

In [19]:
class CIFARClassifier (nn.Module):
  def __init__(self):
    super(CIFARClassifier , self).__init__()
    self.conv1 = nn.Conv2d(3 , 32, kernel_size=3 , padding=1)
    self.conv2 = nn.Conv2d(32 , 64 , kernel_size=3 , padding=1)
    self.conv3 = nn.Conv2d(64 , 128 , kernel_size=3 , padding=1)
    self.pool = nn.MaxPool2d(2,2)
    self.fc1 = nn.Linear(128*4*4 , 512)
    self.fc2 = nn.Linear(512 , 256)
    self.fc3 = nn.Linear(256 , 10)
  def forward(self , x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = self.pool(F.relu(self.conv3(x)))
    x = x.view(-1 , 128*4*4)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x


In [20]:
model = CIFARClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters() , lr=0.001)

In [21]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and being used.")
else:
    device = torch.device("cpu")
    print("No GPU available, using the CPU instead.")

GPU is available and being used.


In [22]:
for epoch in range(10):
  running_loss = 0.0
  model.train()
  model.to(device)
  for i , (images , labels) in enumerate(train_loader):
    images , labels = images.to(device) , labels.to(device)
    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs , labels)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
    if i % 128 == 127:
      print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 100:.3f}')
      running_loss = 0.0
print('Finished Training')

[1,   128] loss: 2.390
[1,   256] loss: 1.977
[1,   384] loss: 1.767
[2,   128] loss: 1.611
[2,   256] loss: 1.516
[2,   384] loss: 1.423
[3,   128] loss: 1.315
[3,   256] loss: 1.264
[3,   384] loss: 1.230
[4,   128] loss: 1.138
[4,   256] loss: 1.124
[4,   384] loss: 1.087
[5,   128] loss: 1.041
[5,   256] loss: 1.003
[5,   384] loss: 0.989
[6,   128] loss: 0.940
[6,   256] loss: 0.951
[6,   384] loss: 0.930
[7,   128] loss: 0.885
[7,   256] loss: 0.882
[7,   384] loss: 0.877
[8,   128] loss: 0.838
[8,   256] loss: 0.836
[8,   384] loss: 0.819
[9,   128] loss: 0.792
[9,   256] loss: 0.780
[9,   384] loss: 0.787
[10,   128] loss: 0.753
[10,   256] loss: 0.758
[10,   384] loss: 0.763
Finished Training


In [23]:
accuracy_metric = Accuracy(task="multiclass", num_classes=10).to(device)
model.eval()
model.to(device)
accuracy_metric.reset()
with torch.no_grad():
    for images , labels in test_loader:
        images , labels = images.to(device) , labels.to(device)
        outputs = model(images)
        accuracy_metric.update(outputs, labels)
final_accuracy = accuracy_metric.compute()
print(f'Accuracy on the test set: {final_accuracy:.2%}')

Accuracy on the test set: 79.25%
